In [ ]:
import json
import pandas as pd
import numpy as np

def load_events(path):
    with open(path) as f:
        data = json.load(f)
    return data["traceEvents"]

def flatten_event(e, source):
    """展开 args，仅保留 name, ts, args.xxx，并增加 from 字段"""

    out = {
        "name": e.get("name"),
        "ts": e.get("ts"),
        "from": source, 
    }

    args = e.get("args", {})
    if isinstance(args, dict):
        for k, v in args.items():
            out[f"args.{k}"] = v

    return out

def preprocess_events(events):
    tmp = pd.DataFrame(events)
    args_uint64 = tmp.dtypes[tmp.dtypes == "float64"]
    print(args_uint64.index.tolist())
    for e in events:
        for arg in args_uint64.index.tolist():
            if arg not in e:
                e[arg] = 0

path_sender = "expr/trace_sender.json"
path_receiver = "expr/trace_receiver.json"

# 读取原始数据
events_sender = load_events(path_sender)
events_receiver = load_events(path_receiver)

# 展开 + 加 from 字段
flat_sender = [flatten_event(e, "sender") for e in events_sender]
flat_receiver = [flatten_event(e, "receiver") for e in events_receiver]

# 合并两个列表
all_events = flat_sender + flat_receiver

preprocess_events(all_events)

df = pd.DataFrame(all_events)

df.dtypes
df["name"].unique().tolist()

['args.last_target_bps', 'args.last_stable_target_bps', 'args.last_fraction_loss_', 'args.last_rtt_ms', 'args.value', 'args.update_target_bitrate_bps', 'args.encoder_target_rate_bps', 'args.protection_bitrate_bps', 'args.encoder_overhead_rate_bps', 'args.post_encode_overhead_bps', 'args.rtp_ts_capture', 'args.rtp_ts', 'args.capture_time_ms', 'args.frame_size', 'args.packet_count', 'args.seq', 'args.transport_seq', 'args.is_received', 'args.last_rtp_ts', 'args.num_media_packets', 'args.num_fec_packets', 'args.param_fec_rate', 'args.fec_payload_type', 'args.fec_ssrc', 'args.protected_frame_rtp_ts', 'args.rtt_ms', 'args.total_delta', 'args.lost_delta', 'args.payload_type', 'args.is_recovered', 'args.picture_id', 'args.render_time_ms', 'args.fec_seq', 'args.scheduled_decode_time_ms']


['BitrateAllocator:OnChanged',
 'rtt_ms',
 'loss',
 'bwe_bps',
 'Sender:AllocateBitrate',
 'Frame:Captured',
 'Frame:Start Encode',
 'Frame:Encoded',
 'Frame:Packetization',
 'Frame:Dropped',
 'Packet:Sent',
 'RTCP:TransportPacketFeedback',
 'Frame:Generate FEC',
 'FlexFEC:GetFecPackets',
 'RTCP:RTT',
 'RTCP:ReportBlocksTransportLossReport',
 'Packet:Received RTP',
 'FlexFEC:Receive RTP Packet',
 'Frame:Received EncodedFrame',
 'Frame:ToFrameBuffer',
 'Frame:ReadytoDecode',
 'Frame:Start Decode',
 'FEC:Receive FEC Packet',
 'Frame:Decoded',
 'Frame:Scheduled for Decode']

In [ ]:
all_frame_rtp_ts_capture = df[df["name"] == "Frame:Captured"]["args.rtp_ts_capture"].unique()
all_frame_rtp_ts_capture.sort()

for rtp_ts_capture in all_frame_rtp_ts_capture:
    print("")
    print("Frame rtp_ts_capture =", rtp_ts_capture)
    event_frame_captured = df[(df["name"] == "Frame:Captured") & (df["args.rtp_ts_capture"] == rtp_ts_capture)]
    event_frame_start_encode = df[(df["name"] == "Frame:Start Encode") & (df["args.rtp_ts_capture"] == rtp_ts_capture)]
    
    if event_frame_start_encode.empty:
        event_frame_dropped = df[(df["name"] == "Frame:Dropped") & (df["args.rtp_ts_capture"] == rtp_ts_capture)]
        reason = event_frame_dropped["args.reason"].values[0] 
        print(f"    dropped before encoding, reason={reason}!")
        continue
    else:
        event_frame_encoded = df[(df["name"] == "Frame:Encoded") & (df["args.rtp_ts_capture"] == rtp_ts_capture)]
        
        if event_frame_encoded.empty:
            print(f"    dropped by encoder!")
            continue
        
        ts_captured = event_frame_captured["ts"].values[0]
        ts_encoded = event_frame_encoded["ts"].values[0]
        rtp_ts = event_frame_encoded["args.rtp_ts"].values[0]
        frame_type = event_frame_encoded["args.frame_type"].values[0]
        frame_size = event_frame_encoded["args.frame_size"].values[0]
        event_frame_packetization = df[(df["name"] == "Frame:Packetization") & (df["args.rtp_ts"] == rtp_ts)]
        packet_count = event_frame_packetization["args.packet_count"].values[0]
        event_frame_generate_fec = df[(df["name"] == "Frame:Generate FEC") & (df["args.last_rtp_ts"] == rtp_ts)]
        num_media_packets = event_frame_generate_fec["args.num_media_packets"].values[0] if not event_frame_generate_fec.empty else 0 
        num_fec_packets = event_frame_generate_fec["args.num_fec_packets"].values[0] if not event_frame_generate_fec.empty else 0
        
        events_packet_sent = df[(df["name"] == "Packet:Sent") & ((df["args.rtp_ts"] == rtp_ts) | (df["args.protected_frame_rtp_ts"] == rtp_ts))] 
        events_packet_sent = events_packet_sent.sort_values(by="ts")
         
        last_packet_sent_ts = events_packet_sent["ts"].values[-1] if not events_packet_sent.empty else None
        
        encoding_delay = ts_encoded - ts_captured
        pacing_delay = last_packet_sent_ts - ts_encoded if last_packet_sent_ts is not None else 0
        sender_side_delay = last_packet_sent_ts - ts_captured if last_packet_sent_ts is not None else None
        
        print(f"rtp_ts={rtp_ts}, frame_type={frame_type}, frame_size={frame_size} bytes, packet_count={packet_count}")
        
        
        for _, packet_event in events_packet_sent.iterrows():
            seq = packet_event["args.seq"]
            packet_type = packet_event["args.packet_type"]
            transport_seq = packet_event["args.transport_seq"]
            print(f"    Packet seq_num={seq}, packet_type={packet_type}, transport_seq={transport_seq}, delay_since_capture={(packet_event['ts'] - ts_captured)/1000} ms")
        
        if last_packet_sent_ts is None:
            print(f"    During sending!")
            continue 
        
        print(f"num_media_packets={num_media_packets}, num_fec_packets={num_fec_packets}")
        print(f"sender_side_delay={sender_side_delay/1000} ms, encoding_delay = {encoding_delay/1000} ms, pacing delay = {pacing_delay/1000} ms")
    



Frame rtp_ts_capture = 3669601574
rtp_ts=1477715563, frame_type=key, frame_size=52321 bytes, packet_count=48, num_media_packets=48, num_fec_packets=1
sender_side_delay=361.062 ms, encoding_delay = 70.996 ms, pacing delay = 290.066 ms
    Packet seq_num=23666, packet_type=video, transport_seq=2, delay_since_capture=74.025 ms
    Packet seq_num=23667, packet_type=video, transport_seq=3, delay_since_capture=85.738 ms
    Packet seq_num=23668, packet_type=video, transport_seq=4, delay_since_capture=95.345 ms
    Packet seq_num=23669, packet_type=video, transport_seq=5, delay_since_capture=105.672 ms
    Packet seq_num=23670, packet_type=video, transport_seq=7, delay_since_capture=116.262 ms
    Packet seq_num=23671, packet_type=video, transport_seq=8, delay_since_capture=122.665 ms
    Packet seq_num=23672, packet_type=video, transport_seq=9, delay_since_capture=128.06 ms
    Packet seq_num=23673, packet_type=video, transport_seq=10, delay_since_capture=133.495 ms
    Packet seq_num=23674

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'